In [4]:
import pandas as pd
import numpy as np
from string import punctuation
import random
from itertools import islice

In [5]:
from nltk import word_tokenize, sent_tokenize, FreqDist, Text
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [7]:
from pymystem3 import Mystem #russian lemmatizer

In [182]:
import matplotlib
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Read Raw Texts

In [9]:
sklepy = open("./sklepy_cynamonowe_polish.txt","r") 
story_text = sklepy.read().split('&&&&&')
story_text.pop(0)

''

In [10]:
book_title = ['Sklepy Cynamonowe'] * 13
story_titles = ['Sierpień','Nawiedzenie','Ptaki','Manekiny','Traktat o Manekinach albo Wtórna Księga Rodzaju',
'Nemrod','Pan','Pan Karol','Sklepy Cynamonowe','Ulica Krokodyli','Karakony','Wichura','Noc Wielkiego Sezonu']
df1 = pd.DataFrame(list(zip(book_title, story_titles, story_text)), columns = ['book', 'story_title', 'text_polish']) 

In [11]:
sanatorium = open("./sanatorium_pod_klepsydra_polish.txt","r") 
story_text = sanatorium.read().split('&&&&&')

In [12]:
book_title = ['Sanatorium Pod Klepysdrą'] * 13
story_titles = ['Księga','Genialna Epoka ','Wiosna','Noc lipcowa','Mój ojciec wstępuje do strażaków',
'Druga jesień','Martwy sezon','Sanatorium pod Klepsydrą','Dodo','Edzio','Emeryt','Samotność',
'Ostatnia ucieczka ojca']
df2 = pd.DataFrame(list(zip(book_title, story_titles, story_text)), columns =['book', 'story_title', 'text_polish']) 

In [13]:
ogdf = pd.concat([df1,df2],ignore_index=True)
df = ogdf

In [14]:
df.head()

,book,story_title,text_polish
0,Sklepy Cynamonowe,Sierpień,\n\nSierpień\n\n1\n\nW lipcu ojciec mój wyjeżd...
1,Sklepy Cynamonowe,Nawiedzenie,\n\nNAWIEDZENIE\n\n1\n\nJuż wówczas miasto nas...
2,Sklepy Cynamonowe,Ptaki,"\n\nPTAKI\n\nNadeszły żółte, pełne nudy dni zi..."
3,Sklepy Cynamonowe,Manekiny,\n\nManekiny\n\nTa ptasia impreza mego ojca by...
4,Sklepy Cynamonowe,Traktat o Manekinach albo Wtórna Księga Rodzaju,\n\nTRAKTAT O MANEKINACH ALBO WTÓRNA KSIĘGA RO...


In [15]:
df.tail()

,book,story_title,text_polish
21,Sanatorium Pod Klepysdrą,Dodo,\n\nPrzychodził do nas w sobotę po południu w ...
22,Sanatorium Pod Klepysdrą,Edzio,"\n\nEDZIO\n\nNa tym samym, co my, piętrze domu..."
23,Sanatorium Pod Klepysdrą,Emeryt,\nEMERYT\n\nJestem emerytem w dosłownym i całk...
24,Sanatorium Pod Klepysdrą,Samotność,\n\nSAMOTNOŚĆ \n\nOd kiedy mogę wychodzić na m...
25,Sanatorium Pod Klepysdrą,Ostatnia ucieczka ojca,\n\nOstatnia ucieczka ojca\n\nByło to w późnym...


### Functions to Process Text Data

#### Basic Text Preprocessing

In [17]:
def tokenize_text(text):
    '''divide body of text into individuals words (tokens) excluding punctuation!
    '''
    tokens = [t for t in word_tokenize(text.lower()) if t not in (',', '“', '”', '"','.', '_', '–', '—')]
    return(tokens)

In [18]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation) # replacing the punctuations with no space
    return text.translate(translator)


Stopwords are commonly found words in language. There is no systematic way of identifying these words.  
There exist several collections of stopwords for different languages.  
The NLTK package supports a collection of stopwords for English. I provide a example below.  
For other languages (Polish, Russian) I found a collection of stopwords on Github. I provide a link in the comments.


In [20]:
#nltk.download('stopwords')
english_stopwords = stopwords.words('english')
english_stopwords = np.array(english_stopwords)
english_stopwords[0:25]

array(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
       "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
       'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
       'hers'], dtype='<U10')

In [21]:
#https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt
polish_stopwords = open("./polish_stopwords.txt","r") 
polish_stopwords = polish_stopwords.readlines()
polish_stopwords = [x.replace('\n', '') for x in polish_stopwords]
polish_stopwords = np.array(polish_stopwords)
polish_stopwords[0:25]

array(['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale',
       'alez', 'ależ', 'ani', 'az', 'aż', 'bardziej', 'bardzo', 'beda',
       'bedzie', 'bez', 'deda', 'będą', 'bede', 'będę', 'będzie', 'bo',
       'bowiem'], dtype='<U12')

In [22]:
#https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.txt
russian_stopwords = open("./russian_stopwords.txt","r") 
russian_stopwords = russian_stopwords.readlines()
russian_stopwords = [x.replace('\n', '') for x in russian_stopwords]
russian_stopwords = np.array(russian_stopwords)
russian_stopwords[0:25]

array(['c', 'а', 'алло', 'без', 'белый', 'близко', 'более', 'больше',
       'большой', 'будем', 'будет', 'будете', 'будешь', 'будто', 'буду',
       'будут', 'будь', 'бы', 'бывает', 'бывь', 'был', 'была', 'были',
       'было', 'быть'], dtype='<U14')

In [23]:
def remove_stopwords(text, stop_words):
    '''removes select stopwords collection i.e. common words that appear frequently like articles, prepositions etc.'''
    text = [word.lower() for word in text.split() if word.lower() not in stop_words]
    return " ".join(text) # join word list with space separator

#### Functions for Lemmatization

******************************

Besides removing stopwords and punctuation, there is another important step in preprocessing text for analysis.  
This step is called normalization. In any language, the same word takes many forms i.e. "smile", "smiles", "smiling", "smiled" in English.  
There are two common ways to standardize language:  

**Stemming**  
Stemming reduces words to their root word ('stem') often by removing any endings or prefixes to the base word.  

**Lemmatization**  
Lemmatization transforms words to a common base word.

A good example from Wikipedia for the English words "produce":  
The lemma is "produce", but the stem is "produc-"

Lemmatization yields better standardization in my opinion because the result of the normalization is itself a real word. "Produc-" isn't a word, by "produce". 

This is really important for Slavic languages like Polish and Russian because a single word can have a number of endings. Linguists call these languages "highly inflected" languages.

You can see the result below. For the base word "jabłko" i.e. "apple", it can take several different forms depending on the contex of the sentence. It can take these forms:

jabłko, jabłku, jabłkom, jabłka, jabłek, jabłkach, jabłkiem, jabłkami

As with stopwords, there are several different solutions for lemmatization for different languages. I use **pymystem3** package for lemmatization in Russian.  

For Polish, I downloaded a dictionary of lemmats from the Polimorf library. I use this dictionary as a lookup table. For each tokenized `word` in the text, I return it's `lemmat` in the dictionary. This allows me to standardize the language and find the root for all words in the text, despite the numerous endings each word can take.

******************************

In [25]:
polimorf = pd.read_csv('./polimorf_dictionary.tab', sep = '\t')
polimorf.drop_duplicates(keep = 'first', inplace = True) 
polimorf.rename(columns={"a": "word", "a.1": "lemmat", "pospolita": "czesc_mowy"}, inplace = True)

In [26]:
polimorf[polimorf['lemmat'] == 'jabłko']

,word,lemmat,interj,czesc_mowy
1201339,jabłku,jabłko,subst:sg:dat:n2,pospolita
1201340,jabłkom,jabłko,subst:pl:dat:n2,pospolita
1201341,jabłka,jabłko,subst:pl:nom:n2,pospolita
1201342,jabłka,jabłko,subst:pl:acc:n2,pospolita
1201343,jabłek,jabłko,subst:pl:gen:n2,pospolita
1201344,jabłko,jabłko,subst:sg:nom:n2,pospolita
1201345,jabłka,jabłko,subst:pl:voc:n2,pospolita
1201346,jabłko,jabłko,subst:sg:voc:n2,pospolita
1201347,jabłkach,jabłko,subst:pl:loc:n2,pospolita
1201348,jabłka,jabłko,subst:sg:gen:n2,pospolita


In [29]:
#these are the parts of speech. unfortunately, the dictionary doesn't distinguish between adj, verb, etc.
polimorf.czesc_mowy.value_counts() 

pospolita               6004695
geograficzna              88476
nazwisko                  72178
imię                      47922
własna                    14754
organizacja                3881
określenie dodatkowe        451
osoba                        73
etnonim                      51
wytwór                       42
wydarzenie                   42
Name: czesc_mowy, dtype: int64

In [30]:
lemmat_dict = dict(zip(polimorf.word, polimorf.lemmat)) 
poc_dict = dict(zip(polimorf.word, polimorf.czesc_mowy)) 

In [31]:
def lemmatize_tokens(tokens):
    '''looks up each token in a dictionary of lemmats and retrieves stem if exists'''
    new_tokens = []
    for token in tokens:
        try:
            new_tokens.append(lemmat_dict[token])
        except:
            new_tokens.append(token)
    return(new_tokens)

#### Retrieve Basic Statistics and Information about Texts

In [32]:
def get_keystrokes(text):
    '''count characters without spaces'''
    kstr = len(text) - text.count(' ')
    return(kstr)

In [33]:
def basic_stats(text):
    '''
    this function returns the number of words, distinct words and the lexical richness.
    lexical richness is just the ratio between the number of distinct words and the number of total words.
    '''
    tokens = tokenize_text(text)
    distinct_tokens = set(tokens)
    lexical_richness = len(distinct_tokens) / len(tokens) # distinct words / all words
    return len(tokens), len(distinct_tokens), lexical_richness

In [34]:
def get_collocation(tokens):
    '''return collocation i.e. word sequences that co-occur more often than by chance'''
    ntext = Text(tokens)
    return ntext.collocation_list()

In [35]:
def advanced_stats(text, num_words, num_hapax, lemmat = False):
    '''
    return top x freq words (num_words), y random hapax (num_hapax), total hapax words, and collocated words.
    hapax legomena = word that occurs only once within a work.
    collocation = sequence of words or terms that co-occur more often than would be expected by chance
    lemmatization is set to False by default.
    '''
    tokens = tokenize_text(text)
    if lemmat:
        tokens = lemmatize_tokens(tokens)
    #take top most frequent words
    freq = FreqDist(tokens)
    top_words = freq.most_common(num_words)
    #take random sample of hapaxes
    all_hapaxes = freq.hapaxes()
    random_hapax = [word for word in random.sample(all_hapaxes, num_hapax)]
    #get all collocated words
    collocats = get_collocation(tokens)
    return dict(top_words), ', '.join(random_hapax), len(all_hapaxes), ', '.join(collocats)

In [36]:
def parts_of_speech(tokens):
    '''looks up each token in a dictionary of lemmats and retrieves part of speech if lemmat exists'''
    new_tokens = []
    for token in tokens:
        try:
            new_tokens.append(poc_dict[token])
        except:
            new_tokens.append(token)
    return(new_tokens)

### Analyze Each Story

In [37]:
df = ogdf

In [38]:
df.head()

,book,story_title,text_polish
0,Sklepy Cynamonowe,Sierpień,\n\nSierpień\n\n1\n\nW lipcu ojciec mój wyjeżd...
1,Sklepy Cynamonowe,Nawiedzenie,\n\nNAWIEDZENIE\n\n1\n\nJuż wówczas miasto nas...
2,Sklepy Cynamonowe,Ptaki,"\n\nPTAKI\n\nNadeszły żółte, pełne nudy dni zi..."
3,Sklepy Cynamonowe,Manekiny,\n\nManekiny\n\nTa ptasia impreza mego ojca by...
4,Sklepy Cynamonowe,Traktat o Manekinach albo Wtórna Księga Rodzaju,\n\nTRAKTAT O MANEKINACH ALBO WTÓRNA KSIĘGA RO...


#### Retrieve Basic Stats for Each Schulz Story (before and after preprocessing)

In [44]:
df['full_length'] = df['text_polish'].apply(len) #return length of text with spaces

In [45]:
df.text_polish[0][0:1500] #before text processing

'\n\nSierpień\n\n1\n\nW lipcu ojciec mój wyjeżdżał do wód i zostawiał mnie z matką i starszym bratem na pastwę białych od żaru i oszołamiających dni letnich. Wertowaliśmy, odurzeni światłem, w tej wielkiej księdze wakacji, której wszystkie karty pałały od blasku i miały na dnie słodki do omdlenia miąższ złotych gruszek.\n\nAdela wracała w świetliste poranki, jak Pomona z ognia dnia rozżagwionego, wysypując z koszyka barwną urodę słońca lśniące, pełne wody pod przejrzystą skórką czereśnie, tajemnicze, czarne wiśnie, których woń przekraczała to, co ziszczało się w smaku; morele, w których miąższu złotym był rdzeń długich popołudni; a obok tej czystej poezji owoców wyładowywała nabrzmiałe siłą i pożywnością płaty mięsa z klawiaturą żeber cielęcych, wodorosty jarzyn, niby zabite głowonogi i meduzy surowy materiał obiadu o smaku jeszcze nie uformowanym i jałowym, wegetatywne i telluryczne ingrediencje obiadu o zapachu dzikim i polnym.\n\nPrzez ciemne mieszkanie na pierwszym piętrze kamienic

In [46]:
df['text_polish'] = df['text_polish'].apply(remove_punctuation)

In [48]:
df.text_polish[0][0:1500] #after removing punctuation

'\n\nSierpień\n\n1\n\nW lipcu ojciec mój wyjeżdżał do wód i zostawiał mnie z matką i starszym bratem na pastwę białych od żaru i oszołamiających dni letnich Wertowaliśmy odurzeni światłem w tej wielkiej księdze wakacji której wszystkie karty pałały od blasku i miały na dnie słodki do omdlenia miąższ złotych gruszek\n\nAdela wracała w świetliste poranki jak Pomona z ognia dnia rozżagwionego wysypując z koszyka barwną urodę słońca lśniące pełne wody pod przejrzystą skórką czereśnie tajemnicze czarne wiśnie których woń przekraczała to co ziszczało się w smaku morele w których miąższu złotym był rdzeń długich popołudni a obok tej czystej poezji owoców wyładowywała nabrzmiałe siłą i pożywnością płaty mięsa z klawiaturą żeber cielęcych wodorosty jarzyn niby zabite głowonogi i meduzy surowy materiał obiadu o smaku jeszcze nie uformowanym i jałowym wegetatywne i telluryczne ingrediencje obiadu o zapachu dzikim i polnym\n\nPrzez ciemne mieszkanie na pierwszym piętrze kamienicy w rynku przechodz

In [49]:
df['length'] = df['text_polish'].apply(len)

In [50]:
df['keystrokes'] = df['text_polish'].apply(get_keystrokes)

In [52]:
df['words'], df['distinct_words'], df['lexical_richness'] = zip(*df['text_polish'].map(basic_stats))

In [53]:
df.head()

,book,story_title,text_polish,full_length,length,keystrokes,words,distinct_words,lexical_richness
0,Sklepy Cynamonowe,Sierpień,\n\nSierpień\n\n1\n\nW lipcu ojciec mój wyjeżd...,15980,15556,13290,2288,1469,0.642045
1,Sklepy Cynamonowe,Nawiedzenie,\n\nNAWIEDZENIE\n\n1\n\nJuż wówczas miasto nas...,12601,12295,10506,1830,1104,0.603279
2,Sklepy Cynamonowe,Ptaki,\n\nPTAKI\n\nNadeszły żółte pełne nudy dni zim...,9081,8851,7569,1286,873,0.678849
3,Sklepy Cynamonowe,Manekiny,\n\nManekiny\n\nTa ptasia impreza mego ojca by...,12420,12126,10357,1780,1157,0.650000
4,Sklepy Cynamonowe,Traktat o Manekinach albo Wtórna Księga Rodzaju,\n\nTRAKTAT O MANEKINACH ALBO WTÓRNA KSIĘGA RO...,19473,18893,16099,2746,1598,0.581937


#### Advanced Stats (before vs after removing stopwords, lemmatization)

In [55]:
df['top_words'], df['hapax_words'], df['total_hapax_words'], df['colloc_words'] = zip(*df['text_polish'].map(lambda w: advanced_stats(w, 10, 10)))

Top words are articles `i`, `w` and the lexical richness is very high across stories.

In [56]:
df.head() #before processing

,book,story_title,text_polish,full_length,length,keystrokes,words,distinct_words,lexical_richness,top_words,hapax_words,total_hapax_words,colloc_words
0,Sklepy Cynamonowe,Sierpień,\n\nSierpień\n\n1\n\nW lipcu ojciec mój wyjeżd...,15980,15556,13290,2288,1469,0.642045,"{'i': 99, 'w': 77, 'się': 59, 'z': 56, 'na': 5...","piliśmy, zawierało, ciepłe, harmonii, dawnych,...",1246,"ciotka agata, dni letnich, jak gdyby, zdawało ..."
1,Sklepy Cynamonowe,Nawiedzenie,\n\nNAWIEDZENIE\n\n1\n\nJuż wówczas miasto nas...,12601,12295,10506,1830,1104,0.603279,"{'i': 91, 'w': 63, 'się': 62, 'z': 45, 'na': 3...","cienia, hałaśliwymi, klepsydry, porzuconych, d...",926,"przez chwilę, mego ojca, nie patrząc, gniew bo..."
2,Sklepy Cynamonowe,Ptaki,\n\nPTAKI\n\nNadeszły żółte pełne nudy dni zim...,9081,8851,7569,1286,873,0.678849,"{'w': 52, 'i': 47, 'się': 37, 'z': 28, 'na': 1...","dzięki, ręki, mogliśmy, przyjęcia, dlań, szafi...",749,"górnych regionach, lamp wiszących, trzepiąc rę..."
3,Sklepy Cynamonowe,Manekiny,\n\nManekiny\n\nTa ptasia impreza mego ojca by...,12420,12126,10357,1780,1157,0.650000,"{'w': 70, 'i': 68, 'się': 46, 'z': 44, 'na': 2...","szukania, móc, popołudnia, cofały, gęstą, myśl...",982,"mój ojciec, godne uwagi, treści więcej, mego o..."
4,Sklepy Cynamonowe,Traktat o Manekinach albo Wtórna Księga Rodzaju,\n\nTRAKTAT O MANEKINACH ALBO WTÓRNA KSIĘGA RO...,19473,18893,16099,2746,1598,0.581937,"{'i': 96, 'w': 85, 'się': 70, 'z': 48, 'na': 4...","włosów, graniczne, nałóg, okazały, nogami, chy...",1322,"moje panie, mój ojciec, ojciec mój, spuszczony..."


In [61]:
df.describe()

,full_length,length,keystrokes,words,distinct_words,lexical_richness,total_hapax_words
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,14991.576923,14595.038462,12441.153846,2162.346154,1289.923077,0.614668,1027.846154
std,7574.573673,7369.332162,6286.614852,1086.463361,565.389489,0.045469,451.727855
min,4469.000000,4323.000000,3625.000000,701.000000,449.000000,0.535849,331.000000
25%,9444.500000,9198.250000,7852.250000,1343.000000,894.000000,0.571548,713.750000
50%,13081.500000,12756.500000,10833.500000,1910.000000,1151.500000,0.618270,918.000000
75%,20688.000000,20135.000000,17245.500000,2926.000000,1732.000000,0.648977,1372.500000
max,32553.000000,31652.000000,26855.000000,4770.000000,2556.000000,0.687982,2014.000000


In [57]:
df['text_polish'] = df['text_polish'].apply(lambda t: remove_stopwords(t, polish_stopwords))

In [58]:
df.text_polish[0][0:1500] #after removing stopwords

'sierpień 1 lipcu ojciec wyjeżdżał wód zostawiał matką starszym bratem pastwę białych żaru oszołamiających dni letnich wertowaliśmy odurzeni światłem wielkiej księdze wakacji karty pałały blasku miały dnie słodki omdlenia miąższ złotych gruszek adela wracała świetliste poranki pomona ognia dnia rozżagwionego wysypując koszyka barwną urodę słońca lśniące pełne wody przejrzystą skórką czereśnie tajemnicze czarne wiśnie woń przekraczała ziszczało smaku morele miąższu złotym rdzeń długich popołudni czystej poezji owoców wyładowywała nabrzmiałe siłą pożywnością płaty mięsa klawiaturą żeber cielęcych wodorosty jarzyn niby zabite głowonogi meduzy surowy materiał obiadu smaku uformowanym jałowym wegetatywne telluryczne ingrediencje obiadu zapachu dzikim polnym ciemne mieszkanie pierwszym piętrze kamienicy rynku przechodziło dzień wskroś całe wielkie lato cisza drgających słojów powietrznych kwadraty blasku śniące żarliwy swój sen podłodze melodia katarynki dobyta najgłębszej złotej żyły dnia t

In [59]:
df['top_words'], df['hapax_words'], df['total_hapax_words'], df['colloc_words'] = zip(*df['text_polish'].map(lambda p: advanced_stats(p, 10, 10)))

There are now fewer hapax words, the lexical richness is lower and the top words are no longer articles.

In [60]:
df.head() #after removing stopwords

,book,story_title,text_polish,full_length,length,keystrokes,words,distinct_words,lexical_richness,top_words,hapax_words,total_hapax_words,colloc_words
0,Sklepy Cynamonowe,Sierpień,sierpień 1 lipcu ojciec wyjeżdżał wód zostawia...,15980,15556,13290,2288,1469,0.642045,"{'dnia': 9, 'twarzy': 6, 'żaru': 5, 'zdawało':...","czyż, ciszą, przykucnięta, rozpustnej, ludzka,...",1210,"rynek pusty, wciąż nowo, ciotka agata, dni let..."
1,Sklepy Cynamonowe,Nawiedzenie,nawiedzenie 1 wówczas miasto popadało coraz ch...,12601,12295,10506,1830,1104,0.603279,"{'nocy': 7, 'ojciec': 7, 'wówczas': 6, 'wśród'...","zewnątrz, wybuchały, wejściu, dziwactwa, zamgl...",888,"mego ojca, gniew boży, pogrążonym zawiłych, uk..."
2,Sklepy Cynamonowe,Ptaki,ptaki nadeszły żółte pełne nudy dni zimowe zru...,9081,8851,7569,1286,873,0.678849,"{'ojciec': 8, 'ojca': 8, 'adela': 5, 'pokoju':...","ognia, bielma, tępymi, dachów, wykluwanie, wra...",705,"górnych regionach, lamp wiszących, trzepiąc rę..."
3,Sklepy Cynamonowe,Manekiny,manekiny ptasia impreza mego ojca ostatnim wyb...,12420,12126,10357,1780,1157,0.650000,"{'ojciec': 8, 'pokoju': 6, 'adela': 6, 'adeli'...","rozbieraliśmy, skrzypiących, wydał, nieudolny,...",937,"godne uwagi, polda paulina, matka mogła, mego ..."
4,Sklepy Cynamonowe,Traktat o Manekinach albo Wtórna Księga Rodzaju,traktat manekinach wtórna księga rodzaju – dem...,19473,18893,16099,2746,1598,0.581937,"{'ojciec': 22, 'materii': 13, 'mówił': 10, 'ży...","sklecone, różowych, wspomnień, szelestem, zost...",1282,"spuszczonymi oczyma, mówił ojciec, ciągnął dal..."


In [62]:
df.describe()

,full_length,length,keystrokes,words,distinct_words,lexical_richness,total_hapax_words
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,14991.576923,14595.038462,12441.153846,2162.346154,1289.923077,0.614668,1027.846154
std,7574.573673,7369.332162,6286.614852,1086.463361,565.389489,0.045469,451.727855
min,4469.000000,4323.000000,3625.000000,701.000000,449.000000,0.535849,331.000000
25%,9444.500000,9198.250000,7852.250000,1343.000000,894.000000,0.571548,713.750000
50%,13081.500000,12756.500000,10833.500000,1910.000000,1151.500000,0.618270,918.000000
75%,20688.000000,20135.000000,17245.500000,2926.000000,1732.000000,0.648977,1372.500000
max,32553.000000,31652.000000,26855.000000,4770.000000,2556.000000,0.687982,2014.000000


In [64]:
df['top_words'], df['hapax_words'], df['total_hapax_words'], df['colloc_words'] = zip(*df['text_polish'].map(lambda p: advanced_stats(p, 10, 10, lemmat = True)))

The top words for stories are now much more standardized.

In [65]:
df.head()

,book,story_title,text_polish,full_length,length,keystrokes,words,distinct_words,lexical_richness,top_words,hapax_words,total_hapax_words,colloc_words
0,Sklepy Cynamonowe,Sierpień,sierpień 1 lipcu ojciec wyjeżdżał wód zostawia...,15980,15556,13290,2288,1469,0.642045,"{'dzień': 19, 'swój': 13, 'twarz': 13, 'wielki...","przymykać, młódź, barwny, oktawa, opowiadać, l...",854,"wciąż nowy, ciotka agata, drugi strona, rynka ..."
1,Sklepy Cynamonowe,Nawiedzenie,nawiedzenie 1 wówczas miasto popadało coraz ch...,12601,12295,10506,1830,1104,0.603279,"{'dzień': 18, 'ojciec': 13, 'noc': 11, 'wielki...","unosić, powoli, nastroszyć, zaczerpnąć, urągać...",634,"cichy gaworzyć, godzina gęsty, mój ojciec, gni..."
2,Sklepy Cynamonowe,Ptaki,ptaki nadeszły żółte pełne nudy dni zimowe zru...,9081,8851,7569,1286,873,0.678849,"{'ojciec': 18, 'ptasi': 11, 'pokój': 9, 'swój'...","rozpadać, żaden, gospodarstwo, mętnożółte, tło...",567,"górny region, lampić wisieć, trzepać ręka, reg..."
3,Sklepy Cynamonowe,Manekiny,manekiny ptasia impreza mego ojca ostatnim wyb...,12420,12126,10357,1780,1157,0.650000,"{'swój': 16, 'ojciec': 12, 'dzień': 11, 'pokój...","należyty, odpłacać, rozważać, rozgospodarowywa...",703,"noc zimowy, godny uwaga, polda paulin, otworzy..."
4,Sklepy Cynamonowe,Traktat o Manekinach albo Wtórna Księga Rodzaju,traktat manekinach wtórna księga rodzaju – dem...,19473,18893,16099,2746,1598,0.581937,"{'ojciec': 28, 'materia': 21, 'oko': 14, 'mówi...","środek, muszy, żałosny, materiał, bejcować, św...",892,"noc zimowy, biedny kuzynka, ciągnąć daleko, tł..."


### Plotting Differences Across Stories

In [77]:
fig = px.bar(df, x='story_title', y='words', color = 'book')
fig.update_layout(
    title={
        'text': "Words by Story",
        'y':0.95,
        'x':0.40,
        'xanchor': 'center',
        'yanchor': 'top'
    })
fig.update_xaxes(title_text='Story Title', tickangle=45, tickfont=dict(family='Rockwell', size=10))
fig.update_yaxes(title_text='Words')
fig.show()

In [78]:
fig = px.bar(df, x='story_title', y='lexical_richness', color = 'book')
fig.update_layout(
    title={
        'text': "Lexical Richness by Story",
        'y':0.95,
        'x':0.40,
        'xanchor': 'center',
        'yanchor': 'top'
    })
fig.update_xaxes(title_text='Lexical Richness', tickangle=45, tickfont=dict(family='Rockwell', size=10))
fig.update_yaxes(title_text='Words')
fig.show()

In [79]:
fig = px.bar(df, x='story_title', y='keystrokes', color = 'book')
fig.update_layout(
    title={
        'text': "Keystrokes by Story",
        'y':0.95,
        'x':0.40,
        'xanchor': 'center',
        'yanchor': 'top'
    })
fig.update_xaxes(title_text='Keystrokes', tickangle=45, tickfont=dict(family='Rockwell', size=10))
fig.update_yaxes(title_text='Words')
fig.show()

In [80]:
fig = px.bar(df, x='story_title', y='total_hapax_words', color = 'book')
fig.update_layout(
    title={
        'text': "Hapax Words by Story",
        'y':0.95,
        'x':0.40,
        'xanchor': 'center',
        'yanchor': 'top'
    })
fig.update_xaxes(title_text='Hapax Words', tickangle=45, tickfont=dict(family='Rockwell', size=10))
fig.update_yaxes(title_text='Words')
fig.show()

### Common and Uncommon Words Across Stories

In [122]:
from collections import Counter

In [82]:
def take(n, iterable):
    '''Return first n items of the iterable as a list'''
    return list(islice(iterable, n))

In [123]:
def get_top_words(df, num_words):
    '''
    Retrieves top words from the dictionary of top words stored in top_words column.
    
    '''
    tuple_list = []
    for story in df.iterrows():
        print("***","Story Title: ", story[1]['story_title'],"|","Book: ", story[1]['book'], "***")
        top_words = ""
        for item in take(num_words, story[1]['top_words'].items()):
            top_words = top_words + "," + item[0]
            tuple_list.append(item)
        print("Top Words: ", top_words[1:len(top_words)])
    wordcount = Counter(elem[0] for elem in tuple_list)
    return wordcount

In [125]:
top_words = get_top_words(df, 5)

*** Story Title:  Sierpień | Book:  Sklepy Cynamonowe ***
Top Words:  dzień,swój,twarz,wielki,złoty
*** Story Title:  Nawiedzenie | Book:  Sklepy Cynamonowe ***
Top Words:  dzień,ojciec,noc,wielki,łóżko
*** Story Title:  Ptaki | Book:  Sklepy Cynamonowe ***
Top Words:  ojciec,ptasi,pokój,swój,wielki
*** Story Title:  Manekiny | Book:  Sklepy Cynamonowe ***
Top Words:  swój,ojciec,dzień,pokój,pełny
*** Story Title:  Traktat o Manekinach albo Wtórna Księga Rodzaju | Book:  Sklepy Cynamonowe ***
Top Words:  ojciec,materia,oko,mówić,swój
*** Story Title:  Nemrod | Book:  Sklepy Cynamonowe ***
Top Words:  żyć,podłoga,swój,nemrod,nowy
*** Story Title:  Pan | Book:  Sklepy Cynamonowe ***
Top Words:  powietrze,ściana,słońce,deska,wysoki
*** Story Title:  Pan Karol | Book:  Sklepy Cynamonowe ***
Top Words:  dzień,pościelić,pusty,woda,karol
*** Story Title:  Sklepy Cynamonowe | Book:  Sklepy Cynamonowe ***
Top Words:  noc,pełny,ulica,swój,wielki
*** Story Title:  Ulica Krokodyli | Book:  Sklepy 

In [134]:
tw_df = pd.DataFrame(list(top_words.items()), columns=['word', 'occurrence'])

In [142]:
tw_df['occurrence'] = tw_df['occurrence'].astype('int')

In [143]:
tw_df = tw_df[tw_df['occurrence'] > 1]

In [159]:
fig = px.bar(tw_df.sort_values(by='occurrence', ascending=False), x='word', y='occurrence')
fig.update_layout(
    title={
        'text': "Reoccurring Top Words by Story",
        'y':0.95,
        'x':0.50,
        'xanchor': 'center',
        'yanchor': 'top'
    })
fig.update_xaxes(title_text='Word', tickangle=45, tickfont=dict(family='Rockwell', size=10))
fig.update_yaxes(title_text='# Times Occurs as Top Word')
fig.show()

### Whole Text Corpus

In [160]:
sklepy = open("./sklepy_cynamonowe_polish.txt","r") 
sklepy = sklepy.read()

In [161]:
sanatorium = open("./sanatorium_pod_klepsydra_polish.txt","r") 
sanatorium = sanatorium.read()

#### Processing

In [302]:
full_text = sklepy + sanatorium

In [303]:
full_text = remove_punctuation(full_text)

In [304]:
full_text = remove_stopwords(full_text, polish_stopwords)

#### Stats

In [165]:
get_keystrokes(full_text) #print keystrokes without stopwords, punctuation

269678

In [168]:
basic_stats(full_text) #print number of words, distinct words, lexical richness

(36509, 18787, 0.5145854446848722)

In [309]:
top_words, hapax_words, total_hapax_words, colloc_words = advanced_stats(text = full_text, num_words = 10, num_hapax = 5, lemmat = True)

In [176]:
tw_df = pd.DataFrame(list(top_words.items()), columns=['word', 'occurrence'])

In [177]:
tw_df['occurrence'] = tw_df['occurrence'].astype('int')

In [178]:
tw_df = tw_df[tw_df['occurrence'] > 1]

In [181]:
fig = px.bar(tw_df.sort_values(by='occurrence', ascending=False), x='word', y='occurrence')
fig.update_layout(
    title={
        'text': "Top Words Across All Stories",
        'y':0.95,
        'x':0.50,
        'xanchor': 'center',
        'yanchor': 'top'
    })
fig.update_xaxes(title_text='Word', tickangle=45, tickfont=dict(family='Rockwell', size=10))
fig.update_yaxes(title_text='# Occurences Across Stories')
fig.show()

In [172]:
colloc_words

'wuj hieronim, anna csillag, wciąż nowy, ulica krokodyli, oba strona, sok malinowy, ciotka retycja, noc lipcowy, mój ojciec, grunt rzecz, noc zimowy, szklany drzwi, subiekt teodor, drugi strona, noc letni, epoka genialny, wyrzucać siebie, przędzalnia tkalnia, rzecz dziwny, jeden drugie'

#### Create Dispersion Plot Function

In [465]:
text_to_comp = list(map(str.lower, ntext))

In [466]:
titles = [x.lower() for x in df.story_title.values]

In [467]:
title_offset = {}
for title in titles:
    try:
        offset = text_to_comp.index(title)
    except:
        title_offset[title] = ''
        continue
    title_offset[title] = offset

In [468]:
title_offset['traktat o manekinach albo wtórna księga rodzaju'] = 4838

title_offset['pan'] = 7316

title_offset['pan karol'] = 7892

title_offset['sklepy cynamonowe'] = 8406

title_offset['ulica krokodyli'] = 10401

title_offset['karakony'] = 12207

title_offset['noc wielkiego sezonu'] = 14013

title_offset['księga'] = 16149

title_offset['genialna epoka '] = 18595

title_offset['noc lipcowa'] = 22453

title_offset['mój ojciec wstępuje do strażaków'] = 23581

title_offset['druga jesień'] = 24917

title_offset['martwy sezon'] = 25679

title_offset['sanatorium pod klepsydrą'] = 28607

title_offset['ostatnia ucieczka ojca'] = 35532

In [469]:
shapes = list()
for i in title_offset.values():
    shapes.append({'type': 'line',
                   'xref': 'x',
                   'yref': 'y',
                   'x0': i,
                   'y0': 0,
                   'x1': i,
                   'y1': 9,
                   'line': dict(color = "LightSalmon", width = 1
            )})

In [473]:
def dispersion_plotly(text, words, ignore_case = False, title="Lexical Dispersion Plot", titles = False):
    text = list(text)
    words.reverse()

    if ignore_case:
        words_to_comp = list(map(str.lower, words))
        text_to_comp = list(map(str.lower, text))
    else:
        words_to_comp = words
        text_to_comp = text

    points = [
        (x, y)
        for x in range(len(text_to_comp))
        for y in range(len(words_to_comp))
        if text_to_comp[x] == words_to_comp[y]
    ]
    if points:
        x, y = list(zip(*points))
    else:
        x = y = ()
        
    fig = px.scatter(
        x = x, y = y, 
        color = y, color_continuous_scale='Viridis',
    )

    fig.update_layout(
        title={
            'text': "Lexical Dispersion",
            'y':0.95,
            'x':0.50,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis = dict(
            tickmode = 'array',
            tickvals = list(range(len(words))),
            ticktext = words
        ),
        coloraxis_showscale=False,
    )
    

    fig.update_xaxes(title_text='Word Offset (All Stories)')
    fig.update_yaxes(title_text='Word')
    
    if titles:
        for shape in shapes:
            fig.add_shape(shape)

    fig.show()


#### Dispersion Plots

In [471]:
tokens = tokenize_text(full_text)
ntext = Text(tokens)

In [475]:
dispersion_plotly(ntext, list(top_words.keys()), titles = True)

In [256]:
characters = ['wuj', 'ciotka', 'ojciec', 'teodor', 'matka', 'adela', 'agata']

In [259]:
dispersion_plotly(ntext, characters)

### Whole Text (in Russian)

In [500]:
sklepy_ru = open("./sklepy_cynamonowe_russian.txt","r") 
sklepy_ru = sklepy_ru.read()

In [501]:
sanatorium_ru = open("./sanatorium_pod_klepsydra_russian.txt","r") 
sanatorium_ru = sanatorium_ru.read()

#### Processing

In [502]:
full_text = sklepy_ru + sanatorium_ru

In [503]:
full_text = remove_punctuation(full_text)

In [504]:
full_text = remove_stopwords(full_text, russian_stopwords)

In [505]:
mystem = Mystem() 

In [506]:
tokens = mystem.lemmatize(full_text.lower())

In [507]:
def process_ru_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    return text

In [508]:
full_ru_text = process_ru_text(full_text)

#### Stats

In [509]:
get_keystrokes(full_ru_text) #print keystrokes without stopwords, punctuation

142879

In [510]:
basic_stats(full_ru_text) #print number of words, distinct words, lexical richness

(18432, 7677, 0.41650390625)

In [511]:
top_words, hapax_words, total_hapax_words, colloc_words = advanced_stats(text = full_ru_text, num_words = 10, num_hapax = 5, lemmat = True)

In [512]:
tw_df = pd.DataFrame(list(top_words.items()), columns=['word', 'occurrence'])

In [513]:
tw_df['occurrence'] = tw_df['occurrence'].astype('int')

In [514]:
tw_df = tw_df[tw_df['occurrence'] > 1]

In [515]:
fig = px.bar(tw_df.sort_values(by='occurrence', ascending=False), x='word', y='occurrence')
fig.update_layout(
    title={
        'text': "Top Words Across All Stories",
        'y':0.95,
        'x':0.50,
        'xanchor': 'center',
        'yanchor': 'top'
    })
fig.update_xaxes(title_text='Word', tickangle=45, tickfont=dict(family='Rockwell', size=10))
fig.update_yaxes(title_text='# Occurences Across Stories')
fig.show()

In [516]:
colloc_words

'дядя иероним, тетка ретиция, анна чиллаг, гениальный эпоха, обращать внимание, подзорный труба, мыльный пузырь, переводной картинка, коричный лавка, старший брат, городской площадь, сбивать толк, трактат манекен, пан кароль, дикий сирень, хлопание крыло, дейзи дейзи, польд паулина, тетка перазие, боско милан'

#### Dispersion Plots

In [517]:
tokens = tokenize_text(full_ru_text)
ntext = Text(tokens)

In [518]:
dispersion_plotly(ntext, list(top_words.keys()))

In [526]:
characters = ['дядя', 'тетка', 'отец', 'мать', 'пан', 'аделя', 'агата']

In [527]:
dispersion_plotly(ntext, characters)